# Recommendation Engine for Movies and TV

In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sklearn
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [110]:
data = pd.read_csv("movie_data.csv")
data

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,principal_names,director_names,writer_names,feature_soup
0,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography",6.0,1006,"Elizabeth Tait, John Tait, Nicholas Brierley, ...",Charles Tait,Charles Tait,"Action,Adventure,Biography Elizabeth Tait, Joh..."
1,tt0002130,movie,Dante's Inferno,L'inferno,0,1911,\N,71,"Adventure,Drama,Fantasy",7.0,3857,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",NaN,Dante Alighieri,"Adventure,Drama,Fantasy Salvatore Papa, Arturo..."
2,tt0002423,movie,Passion,Madame DuBarry,0,1919,\N,113,"Biography,Drama,Romance",6.6,1083,"Pola Negri, Emil Jannings, Harry Liedtke, Edua...",Ernst Lubitsch,NaN,"Biography,Drama,Romance Pola Negri, Emil Janni..."
3,tt0002844,movie,Fantômas: In the Shadow of the Guillotine,Fantômas I: À l'ombre de la guillotine,0,1913,\N,54,"Crime,Drama",6.9,2647,"René Navarre, René Navarre, Edmund Breon, Geor...",Louis Feuillade,NaN,"Crime,Drama René Navarre, René Navarre, Edmund..."
4,tt0003014,movie,Ingeborg Holm,Ingeborg Holm,0,1913,\N,96,Drama,7.0,1542,"Hilda Borgström, Georg Grönroos, Aron Lindgren...",Victor Sjöström,NaN,"Drama Hilda Borgström, Georg Grönroos, Aron Li..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93541,tt9915898,tvEpisode,Time for Farewell! Pudding's One Last Request!,Time for Farewell! Pudding's One Last Request!,0,2019,\N,24,"Action,Adventure,Animation",9.4,4259,"Mayumi Tanaka, Tom Henry, Tom Henry, Masako Ka...",Toshinori Fukuzawa,Shôji Yonemura,"Action,Adventure,Animation Mayumi Tanaka, Tom ..."
93542,tt9915906,tvEpisode,The World in Shock! The Fifth Emperor of the S...,The World in Shock! The Fifth Emperor of the S...,0,2019,\N,\N,"Action,Adventure,Animation",8.6,1994,"Mayumi Tanaka, Eric Cherry, Tom Henry, Tom Henry",NaN,NaN,"Action,Adventure,Animation Mayumi Tanaka, Eric..."
93543,tt9916128,tvSeries,Salt City (India),Salt City (India),0,2022,\N,\N,Drama,6.0,3422,"Divyendu Sharma, Piyush Mishra, Monica Chaudha...",NaN,NaN,"Drama Divyendu Sharma, Piyush Mishra, Monica C..."
93544,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,\N,84,Thriller,5.8,1512,"Sergio Castellitto, Lorenzo Richelmy, Anna Fog...",Giacomo Cimini,NaN,"Thriller Sergio Castellitto, Lorenzo Richelmy,..."


In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93546 entries, 0 to 93545
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           93546 non-null  object 
 1   titleType        93546 non-null  object 
 2   primaryTitle     93546 non-null  object 
 3   originalTitle    93546 non-null  object 
 4   isAdult          93546 non-null  int64  
 5   startYear        93546 non-null  int64  
 6   endYear          93546 non-null  object 
 7   runtimeMinutes   93546 non-null  object 
 8   genres           93546 non-null  object 
 9   averageRating    93546 non-null  float64
 10  numVotes         93546 non-null  int64  
 11  principal_names  93480 non-null  object 
 12  director_names   74543 non-null  object 
 13  writer_names     20670 non-null  object 
 14  feature_soup     93546 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 10.7+ MB


In [112]:
data = data[~data["genres"].str.contains(r"\\N")]

In [46]:
data.sort_values(by=["startYear", "numVotes", "averageRating"], ascending=False)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,principal_names,director_names,writer_names,feature_soup
83141,tt6208148,movie,Snow White,Snow White,0,2025,\N,109,"Adventure,Family,Fantasy",2.1,381185,"Rachel Zegler, Emilia Faucher, Gal Gadot, Andr...",Marc Webb,NaN,"Adventure,Family,Fantasy Rachel Zegler, Emilia..."
69118,tt31806037,tvMiniSeries,Adolescence,Adolescence,0,2025,2025,60,"Crime,Drama,Thriller",8.2,216462,"Owen Cooper, Stephen Graham, Ashley Walters, F...",Philip Barantini,NaN,"Crime,Drama,Thriller Owen Cooper, Stephen Grah..."
68606,tt31193180,movie,Sinners,Sinners,0,2025,\N,137,"Action,Drama,Horror",7.7,214542,"Michael B. Jordan, Michael B. Jordan, Miles Ca...",Ryan Coogler,Ryan Coogler,"Action,Drama,Horror Michael B. Jordan, Michael..."
41155,tt12299608,movie,Mickey 17,Mickey 17,0,2025,\N,137,"Adventure,Comedy,Fantasy",6.8,172840,"Robert Pattinson, Steven Yeun, Michael Monroe,...",Bong Joon Ho,NaN,"Adventure,Comedy,Fantasy Robert Pattinson, Ste..."
47423,tt14513804,movie,Captain America: Brave New World,Captain America: Brave New World,0,2025,\N,118,"Action,Adventure,Sci-Fi",5.6,149443,"Anthony Mackie, Anthony Mackie, Harrison Ford,...",Julius Onah,NaN,"Action,Adventure,Sci-Fi Anthony Mackie, Anthon..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,tt0003037,movie,Fantomas: The Man in Black,Fantômas II: Juve contre Fantômas,0,1913,\N,61,"Crime,Drama",6.9,1797,"René Navarre, René Navarre, René Navarre, Edmu...",Louis Feuillade,NaN,"Crime,Drama René Navarre, René Navarre, René N..."
4,tt0003014,movie,Ingeborg Holm,Ingeborg Holm,0,1913,\N,96,Drama,7.0,1542,"Hilda Borgström, Georg Grönroos, Aron Lindgren...",Victor Sjöström,NaN,"Drama Hilda Borgström, Georg Grönroos, Aron Li..."
6,tt0003165,movie,Fantômas: The Dead Man Who Killed,Fantômas III: Le mort qui tue,0,1913,\N,90,"Crime,Drama,Mystery",6.9,1433,"René Navarre, René Navarre, Edmund Breon, Geor...",Louis Feuillade,NaN,"Crime,Drama,Mystery René Navarre, René Navarre..."
1,tt0002130,movie,Dante's Inferno,L'inferno,0,1911,\N,71,"Adventure,Drama,Fantasy",7.0,3857,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",NaN,Dante Alighieri,"Adventure,Drama,Fantasy Salvatore Papa, Arturo..."


In [113]:
processed_data = data.copy()

### Feature Engineering

In [114]:
safe_split = lambda x: x.split(",") if isinstance(x, str) else []

# Apply this function to each of the relevant columns
processed_data['genres_list'] = processed_data['genres'].apply(safe_split)
processed_data['actors_list'] = processed_data['principal_names'].apply(safe_split)
processed_data['directors_list'] = processed_data['director_names'].apply(safe_split)
processed_data['writers_list'] = processed_data['writer_names'].apply(safe_split)

# Let's look at the first row of our new columns to see the result
processed_data[['primaryTitle', 'genres_list', 'actors_list', 'directors_list', 'writers_list']]

,primaryTitle,genres_list,actors_list,directors_list,writers_list
0,The Story of the Kelly Gang,"[Action, Adventure, Biography]","[Elizabeth Tait, John Tait, Nicholas Brierle...",[Charles Tait],[Charles Tait]
1,Dante's Inferno,"[Adventure, Drama, Fantasy]","[Salvatore Papa, Arturo Pirovano, Giuseppe d...",[],[Dante Alighieri]
2,Passion,"[Biography, Drama, Romance]","[Pola Negri, Emil Jannings, Harry Liedtke, ...",[Ernst Lubitsch],[]
3,Fantômas: In the Shadow of the Guillotine,"[Crime, Drama]","[René Navarre, René Navarre, Edmund Breon, ...",[Louis Feuillade],[]
4,Ingeborg Holm,[Drama],"[Hilda Borgström, Georg Grönroos, Aron Lindg...",[Victor Sjöström],[]
...,...,...,...,...,...
93541,Time for Farewell! Pudding's One Last Request!,"[Action, Adventure, Animation]","[Mayumi Tanaka, Tom Henry, Tom Henry, Masak...",[Toshinori Fukuzawa],[Shôji Yonemura]
93542,The World in Shock! The Fifth Emperor of the S...,"[Action, Adventure, Animation]","[Mayumi Tanaka, Eric Cherry, Tom Henry, Tom...",[],[]
93543,Salt City (India),[Drama],"[Divyendu Sharma, Piyush Mishra, Monica Chau...",[],[]
93544,Il talento del calabrone,[Thriller],"[Sergio Castellitto, Lorenzo Richelmy, Anna ...",[Giacomo Cimini],[]


In [49]:
# Let's look at the first row of our new columns to see the result
processed_data[['primaryTitle', 'genres_list', 'actors_list', 'directors_list', 'writers_list']]

,primaryTitle,genres_list,actors_list,directors_list,writers_list
0,The Story of the Kelly Gang,"[Action, Adventure, Biography]","[Elizabeth Tait, John Tait, Nicholas Brierle...",[Charles Tait],[Charles Tait]
1,Dante's Inferno,"[Adventure, Drama, Fantasy]","[Salvatore Papa, Arturo Pirovano, Giuseppe d...",[],[Dante Alighieri]
2,Passion,"[Biography, Drama, Romance]","[Pola Negri, Emil Jannings, Harry Liedtke, ...",[Ernst Lubitsch],[]
3,Fantômas: In the Shadow of the Guillotine,"[Crime, Drama]","[René Navarre, René Navarre, Edmund Breon, ...",[Louis Feuillade],[]
4,Ingeborg Holm,[Drama],"[Hilda Borgström, Georg Grönroos, Aron Lindg...",[Victor Sjöström],[]
...,...,...,...,...,...
93541,Time for Farewell! Pudding's One Last Request!,"[Action, Adventure, Animation]","[Mayumi Tanaka, Tom Henry, Tom Henry, Masak...",[Toshinori Fukuzawa],[Shôji Yonemura]
93542,The World in Shock! The Fifth Emperor of the S...,"[Action, Adventure, Animation]","[Mayumi Tanaka, Eric Cherry, Tom Henry, Tom...",[],[]
93543,Salt City (India),[Drama],"[Divyendu Sharma, Piyush Mishra, Monica Chau...",[],[]
93544,Il talento del calabrone,[Thriller],"[Sergio Castellitto, Lorenzo Richelmy, Anna ...",[Giacomo Cimini],[]


In [115]:
# Function to clean a list of names/genres
def clean_list(input_list):
    # For genres, we just strip whitespace and convert to lowercase
    if not input_list or all(item.isalpha() for item in input_list): # A simple check for genre lists
        return [item.strip().lower() for item in input_list]
    # For names, we also remove spaces within the name
    else:
        return [item.strip().replace(' ', '') for item in input_list]

In [116]:
# Apply the cleaning function to each list column
processed_data['genres_list'] = processed_data['genres_list'].apply(clean_list)
processed_data['actors_list'] = processed_data['actors_list'].apply(clean_list)
processed_data['directors_list'] = processed_data['directors_list'].apply(clean_list)
processed_data['writers_list'] = processed_data['writers_list'].apply(clean_list)

In [117]:
sinners_row = processed_data[processed_data['primaryTitle'] == 'Sinners']
print("Cleaned Actors List for 'Sinners':")
print(sinners_row['actors_list'].iloc[0])

Cleaned Actors List for 'Sinners':
['MichaelB.Jordan', 'MichaelB.Jordan', 'MilesCaton', 'SaulWilliams', 'AndreneWard-Hammond', "JackO'Connell", 'TenajL.Jackson', 'DaveMaldonado', 'AadynEncalarde', 'HelenaHu', 'Yao', 'RyanCoogler', 'RyanCoogler']


### Approach One 

- This method will see the performance of the recommendation matrix if all the features are coupled together in a single string of features

In [118]:

def create_feature_soup(row):
    # This is the key change: dict.fromkeys preserves order while removing duplicates
    ordered_unique_actors = list(dict.fromkeys(row['actors_list']))
    ordered_unique_directors = list(dict.fromkeys(row['directors_list']))
    ordered_unique_writers = list(dict.fromkeys(row['writers_list']))

    # Apply weights to the correctly ordered lists
    # Director gets 3x weight
    # Top 3 actors get 2x weight
    # Writers get 1x weight
    weighted_directors = ordered_unique_directors * 3
    weighted_actors = ordered_unique_actors[:3] * 2  # Now this correctly gets the top 3
    
    # Combine everything into a single list
    soup_list = row['genres_list'] + weighted_directors + weighted_actors + ordered_unique_writers
    
    # Join the list into a single string
    return ' '.join(soup_list)


In [119]:
# Apply this function to the DataFrame to create the final column
processed_data['feature_soup'] = processed_data.apply(create_feature_soup, axis=1)

In [120]:
print("Final Feature Soup for 'Sinners':")
print(processed_data[processed_data['primaryTitle'] == 'The Dark Knight']['feature_soup'].iloc[0])

Final Feature Soup for 'Sinners':
action crime drama ChristopherNolan ChristopherNolan ChristopherNolan ChristianBale HeathLedger AaronEckhart ChristianBale HeathLedger AaronEckhart


In [41]:
processed_data[processed_data['primaryTitle'] == 'Sinners']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,principal_names,director_names,writer_names,feature_soup,genres_list,actors_list,directors_list,writers_list
68606,tt31193180,movie,Sinners,Sinners,0,2025,\N,137,"Action,Drama,Horror",7.7,214542,"Michael B. Jordan, Michael B. Jordan, Miles Ca...",Ryan Coogler,Ryan Coogler,action drama horror RyanCoogler RyanCoogler Ry...,"[action, drama, horror]","[MichaelB.Jordan, MichaelB.Jordan, MilesCaton,...",[RyanCoogler],[RyanCoogler]


#### Vectorization

In [121]:
# stop_words='english' will remove common English words like 'a', 'the', 'is'.
# While our soup is clean, this is a standard best practice.
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(processed_data['feature_soup'])

print("Shape of the TF-IDF matrix:")
print(tfidf_matrix.shape)

Shape of the TF-IDF matrix:
(93493, 96144)


#### Similarity Matrix

In [122]:
indices = pd.Series(processed_data.index, index=processed_data['primaryTitle']).drop_duplicates()

In [73]:
processed_data[indices.index.str.contains("The Social Network")]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,principal_names,director_names,writer_names,feature_soup,genres_list,actors_list,directors_list,writers_list
42658,tt1285016,movie,The Social Network,The Social Network,0,2010,\N,120,"Biography,Drama",7.8,795604,"Jesse Eisenberg, Andrew Garfield, Justin Timbe...",David Fincher,NaN,biography drama DavidFincher DavidFincher Davi...,"[biography, drama]","[JesseEisenberg, AndrewGarfield, JustinTimberl...",[DavidFincher],[]


#### Recommendation

In [123]:
# Step 2: Build the Recommendation Function
def get_recommendations(title, tfidf_matrix):
    # Get the index of the movie that matches the title
    try:
        idx = indices[title]
    except KeyError:
        return "Movie not found in the dataset."

    # Step 2a: Calculate cosine similarity for JUST that one movie
    # We compare the tfidf_matrix[idx] (a single row) with the entire matrix
    cosine_sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)

    # Step 2b: Get the similarity scores of all movies with that movie
    # cosine_sim_scores is a 2D array, so we grab the first (and only) row
    sim_scores = list(enumerate(cosine_sim_scores[0]))

    # Step 2c: Sort the movies based on the similarity scores
    # We sort in descending order (highest similarity first)
    # The key=lambda x: x[1] tells sort to use the similarity score (the 2nd element)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Step 2d: Get the scores of the 10 most similar movies
    # We start from index 1 to skip the movie itself (which will have a score of 1.0)
    sim_scores = sim_scores[1:11]

    # Step 2e: Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Step 2f: Return the titles of the top 10 most similar movies
    return processed_data['primaryTitle'].iloc[movie_indices]

In [124]:
get_recommendations('Oppenheimer', tfidf_matrix)

19372                 The Last Kiss
43963                 Kiss Me Again
63652           Fathers & Daughters
31498                  Seven Pounds
24308      The Pursuit of Happyness
92458                The Best Years
20919          Remember Me, My Love
85940    There's No Place Like Home
17422        But Forever in My Mind
77066                    Summertime
Name: primaryTitle, dtype: object

In [125]:
def get_smarter_recommendations(title, tfidf_matrix, df):
    # Get the index of the movie that matches the title
    try:
        idx = indices[title]
    except KeyError:
        return "Movie not found in the dataset."

    # --- Part 1: Get Raw Similarity Scores ---
    cosine_sim_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim_scores[0]))

    # --- Part 2: Create a DataFrame of Potential Recommendations ---
    # Sort the movies based on similarity, but take a larger pool (e.g., top 250)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:251]
    
    # Get the indices and scores of these top movies
    movie_indices = [i[0] for i in sim_scores]
    content_scores = [i[1] for i in sim_scores]

    # Create a new DataFrame with this pool of recommendations
    recs_df = df.iloc[movie_indices].copy()
    recs_df['content_score'] = content_scores
    
    # --- Part 3: Calculate a Popularity/Quality Score ---
    # We will use a simple weighted rating formula.
    # This is a common method to balance average rating with the number of votes.
    m = df['numVotes'].quantile(0.60) # Minimum votes required to be considered
    C = df['averageRating'].mean()    # The mean rating across the whole dataset

    def weighted_rating(x, m=m, C=C):
        v = x['numVotes']
        R = x['averageRating']
        # Calculation based on the IMDB formula
        return (v / (v + m) * R) + (m / (v + m) * C)

    recs_df['popularity_score'] = recs_df.apply(weighted_rating, axis=1)

    # --- Part 4: Calculate the Final Hybrid Score ---
    # We can tune these weights. Let's start with 70% content, 30% popularity.
    w_content = 0.7
    w_popularity = 0.3
    
    # Normalize the scores to be between 0 and 1 so they can be combined
    recs_df['content_score'] = recs_df['content_score'] / recs_df['content_score'].max()
    recs_df['popularity_score'] = recs_df['popularity_score'] / recs_df['popularity_score'].max()

    recs_df['final_score'] = (w_content * recs_df['content_score']) + \
                             (w_popularity * recs_df['popularity_score'])
    
    # --- Part 5: Sort by Final Score and Return Top Results ---
    final_recs = recs_df.sort_values('final_score', ascending=False).head(10)
    
    return final_recs[['primaryTitle', 'startYear', 'averageRating', 'final_score']]

In [126]:
get_smarter_recommendations('Oppenheimer', tfidf_matrix, processed_data)

,primaryTitle,startYear,averageRating,final_score
19372,The Last Kiss,2001,6.9,0.936340
43963,Kiss Me Again,2010,5.9,0.922261
31498,Seven Pounds,2008,7.6,0.879122
24308,The Pursuit of Happyness,2006,8.0,0.872665
63652,Fathers & Daughters,2015,7.0,0.862506
92458,The Best Years,2020,6.7,0.832217
20919,"Remember Me, My Love",2003,6.4,0.808331
85940,There's No Place Like Home,2018,6.1,0.799951
17422,But Forever in My Mind,1999,6.3,0.793616
77066,Summertime,2016,6.2,0.782132


This method performs badly due to the presence of noise in the overall matrix, movies like Oppenheimer are difficult to recommend off of because it prioritizes anything drama.

### Approach Two

- This Method will use two separate and independent feature sets and then combine their recommendations and see it's performance

In [127]:
# People Soup: Heavily weighted towards the director and main actor
def create_people_soup(row):
    ordered_unique_directors = list(dict.fromkeys(row['directors_list']))
    ordered_unique_actors = list(dict.fromkeys(row['actors_list']))
    
    # Give massive weight to the director and lead actor to emphasize their importance
    weighted_directors = ordered_unique_directors * 5 
    weighted_actors = ordered_unique_actors[:1] * 3 # Just the lead actor
    
    return ' '.join(weighted_directors + weighted_actors)

# Genre Soup: Just the genres
def create_genre_soup(row):
    return ' '.join(row['genres_list'])

processed_data['people_soup'] = processed_data.apply(create_people_soup, axis=1)
processed_data['genre_soup'] = processed_data.apply(create_genre_soup, axis=1)

In [137]:
processed_data[processed_data["primaryTitle"] == "Playing for Keeps"]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,...,principal_names,director_names,writer_names,feature_soup,genres_list,actors_list,directors_list,writers_list,people_soup,genre_soup
49988,tt1540128,movie,Playing for Keeps,Playing for Keeps,0,2012,\N,105,"Comedy,Romance,Sport",5.7,...,"Gerard Butler, Jessica Biel, Dennis Quaid, Noa...",Gabriele Muccino,Robbie Fox,comedy romance sport GabrieleMuccino GabrieleM...,"[comedy, romance, sport]","[GerardButler, JessicaBiel, DennisQuaid, NoahL...",[GabrieleMuccino],[RobbieFox],GabrieleMuccino GabrieleMuccino GabrieleMuccin...,comedy romance sport


#### Vectorization

In [129]:
# People Model
people_tfidf = TfidfVectorizer(stop_words='english')
people_tfidf_matrix = people_tfidf.fit_transform(processed_data['people_soup'])

# Genre Model
genre_tfidf = TfidfVectorizer() # No stop words needed for genres
genre_tfidf_matrix = genre_tfidf.fit_transform(processed_data['genre_soup'])

In [130]:
def get_truly_hybrid_recommendations(title, df, people_matrix, genre_matrix):
    try:
        idx = indices[title]
    except KeyError:
        return "Movie not found in the dataset."

    # --- Calculate Similarity Scores from BOTH models ---
    people_sim_scores = cosine_similarity(people_matrix[idx], people_matrix)[0]
    genre_sim_scores = cosine_similarity(genre_matrix[idx], genre_matrix)[0]
    
    # --- Combine the scores with weights ---
    # This is our key control knob. We're saying people are more important.
    w_people = 0.75 
    w_genre = 0.25
    
    combined_content_scores = (w_people * people_sim_scores) + (w_genre * genre_sim_scores)

    # Now we build the recommendation frame as before, but with the new combined score
    sim_scores = list(enumerate(combined_content_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:251]
    
    movie_indices = [i[0] for i in sim_scores]
    content_scores = [i[1] for i in sim_scores]

    recs_df = df.iloc[movie_indices].copy()
    recs_df['content_score'] = content_scores
    
    # --- Apply the Popularity Filter (as before) ---
    m = df['numVotes'].quantile(0.60)
    C = df['averageRating'].mean()
    def weighted_rating(x, m=m, C=C):
        v = x['numVotes']
        R = x['averageRating']
        return (v / (v + m) * R) + (m / (v + m) * C)

    recs_df['popularity_score'] = recs_df.apply(weighted_rating, axis=1)

    # --- Final Score Calculation ---
    w_content = 0.8 # Let's value our sophisticated content score more
    w_popularity = 0.2
    
    recs_df['content_score'] = recs_df['content_score'] / recs_df['content_score'].max()
    recs_df['popularity_score'] = recs_df['popularity_score'] / recs_df['popularity_score'].max()
    recs_df['final_score'] = (w_content * recs_df['content_score']) + (w_popularity * recs_df['popularity_score'])
    
    final_recs = recs_df.sort_values('final_score', ascending=False).head(10)
    
    return final_recs[['primaryTitle', 'startYear', 'averageRating', 'genres']]

In [131]:
get_truly_hybrid_recommendations('Oppenheimer', processed_data, people_tfidf_matrix, genre_tfidf_matrix)

,primaryTitle,startYear,averageRating,genres
19372,The Last Kiss,2001,6.9,"Comedy,Drama,Romance"
85940,There's No Place Like Home,2018,6.1,"Comedy,Drama,Romance"
20919,"Remember Me, My Love",2003,6.4,"Comedy,Drama,Romance"
43963,Kiss Me Again,2010,5.9,"Drama,Romance"
17422,But Forever in My Mind,1999,6.3,Comedy
24308,The Pursuit of Happyness,2006,8.0,"Biography,Drama"
31498,Seven Pounds,2008,7.6,"Drama,Mystery,Thriller"
63652,Fathers & Daughters,2015,7.0,Drama
92458,The Best Years,2020,6.7,Drama
77066,Summertime,2016,6.2,Drama


#### Debugging

In [132]:
def create_people_soup(row):
    ordered_unique_directors = list(dict.fromkeys(row['directors_list']))
    ordered_unique_actors = list(dict.fromkeys(row['actors_list']))
    weighted_directors = ordered_unique_directors * 5 
    weighted_actors = ordered_unique_actors[:1] * 3
    return ' '.join(weighted_directors + weighted_actors)

In [133]:
def create_genre_soup(row):
    return ' '.join(row['genres_list'])

processed_data['people_soup'] = processed_data.apply(create_people_soup, axis=1)
processed_data['genre_soup'] = processed_data.apply(create_genre_soup, axis=1)

In [134]:
people_tfidf = TfidfVectorizer(stop_words='english')
people_tfidf_matrix = people_tfidf.fit_transform(processed_data['people_soup'])

genre_tfidf = TfidfVectorizer()
genre_tfidf_matrix = genre_tfidf.fit_transform(processed_data['genre_soup'])

In [135]:
# --- Create the soups and matrices as before ---







# We need the indices mapping as well
indices = pd.Series(processed_data.index, index=processed_data['primaryTitle']).drop_duplicates()


# --- THE ALL-NEW DEBUGGING RECOMMENDATION FUNCTION ---

def get_recommendations_with_debugging(title, df, people_matrix, genre_matrix):
    try:
        idx = indices[title]
    except KeyError:
        return "Movie not found in the dataset."

    print(f"\n--- DEBUG: Finding recommendations for '{title}' (index: {idx}) ---")
    
    # --- CHECKPOINT 1: Are the source soups correct? ---
    print("\nDEBUG: People Soup:", df.iloc[idx]['people_soup'])
    print("DEBUG: Genre Soup:", df.iloc[idx]['genre_soup'])

    # --- CHECKPOINT 2: Are the two models producing different results? ---
    people_sim_scores = cosine_similarity(people_matrix[idx], people_matrix)[0]
    genre_sim_scores = cosine_similarity(genre_matrix[idx], genre_matrix)[0]

    # Let's see the top 5 raw recommendations from the PEOPLE model
    people_recs_raw = sorted(list(enumerate(people_sim_scores)), key=lambda x: x[1], reverse=True)[1:6]
    people_recs_indices = [i[0] for i in people_recs_raw]
    print("\n--- DEBUG: Top 5 People-Based Recommendations (Raw) ---")
    print(df['primaryTitle'].iloc[people_recs_indices])

    # Let's see the top 5 raw recommendations from the GENRE model
    genre_recs_raw = sorted(list(enumerate(genre_sim_scores)), key=lambda x: x[1], reverse=True)[1:6]
    genre_recs_indices = [i[0] for i in genre_recs_raw]
    print("\n--- DEBUG: Top 5 Genre-Based Recommendations (Raw) ---")
    print(df['primaryTitle'].iloc[genre_recs_indices])

    # --- CHECKPOINT 3: Is the combination working? ---
    w_people = 0.75 
    w_genre = 0.25
    combined_content_scores = (w_people * people_sim_scores) + (w_genre * genre_sim_scores)
    
    combined_recs_raw = sorted(list(enumerate(combined_content_scores)), key=lambda x: x[1], reverse=True)[1:6]
    combined_recs_indices = [i[0] for i in combined_recs_raw]
    print("\n--- DEBUG: Top 5 Combined-Score Recommendations (Before Popularity Filter) ---")
    print(df['primaryTitle'].iloc[combined_recs_indices])
    
    # --- The rest of the function remains the same ---
    sim_scores = list(enumerate(combined_content_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:251]
    
    movie_indices = [i[0] for i in sim_scores]
    content_scores = [i[1] for i in sim_scores]

    recs_df = df.iloc[movie_indices].copy()
    recs_df['content_score'] = content_scores
    
    m = df['numVotes'].quantile(0.60)
    C = df['averageRating'].mean()
    def weighted_rating(x, m=m, C=C):
        v = x['numVotes']
        R = x['averageRating']
        return (v / (v + m) * R) + (m / (v + m) * C)
    recs_df['popularity_score'] = recs_df.apply(weighted_rating, axis=1)

    w_content = 0.8
    w_popularity = 0.2
    recs_df['content_score'] = recs_df['content_score'] / recs_df['content_score'].max()
    recs_df['popularity_score'] = recs_df['popularity_score'] / recs_df['popularity_score'].max()
    recs_df['final_score'] = (w_content * recs_df['content_score']) + (w_popularity * recs_df['popularity_score'])
    
    final_recs = recs_df.sort_values('final_score', ascending=False).head(10)
    
    print("\n--- FINAL OUTPUT ---")
    return final_recs[['primaryTitle', 'startYear', 'averageRating', 'genres']]



In [136]:
# --- Test the new debugging function ---
get_recommendations_with_debugging('Oppenheimer', processed_data, people_tfidf_matrix, genre_tfidf_matrix)


--- DEBUG: Finding recommendations for 'Oppenheimer' (index: 49978) ---

DEBUG: People Soup: GabrieleMuccino GabrieleMuccino GabrieleMuccino GabrieleMuccino GabrieleMuccino GerardButler GerardButler GerardButler
DEBUG: Genre Soup: comedy romance sport

--- DEBUG: Top 5 People-Based Recommendations (Raw) ---
24308    The Pursuit of Happyness
31498                Seven Pounds
63652         Fathers & Daughters
19372               The Last Kiss
43963               Kiss Me Again
Name: primaryTitle, dtype: object

--- DEBUG: Top 5 Genre-Based Recommendations (Raw) ---
238              The Smart Set
2541              Pat and Mike
6153    The Troubles of Alfred
8107                Semi-Tough
8316            The Main Event
Name: primaryTitle, dtype: object

--- DEBUG: Top 5 Combined-Score Recommendations (Before Popularity Filter) ---
19372                 The Last Kiss
85940    There's No Place Like Home
20919          Remember Me, My Love
43963                 Kiss Me Again
17422        But 

,primaryTitle,startYear,averageRating,genres
19372,The Last Kiss,2001,6.9,"Comedy,Drama,Romance"
85940,There's No Place Like Home,2018,6.1,"Comedy,Drama,Romance"
20919,"Remember Me, My Love",2003,6.4,"Comedy,Drama,Romance"
43963,Kiss Me Again,2010,5.9,"Drama,Romance"
17422,But Forever in My Mind,1999,6.3,Comedy
24308,The Pursuit of Happyness,2006,8.0,"Biography,Drama"
31498,Seven Pounds,2008,7.6,"Drama,Mystery,Thriller"
63652,Fathers & Daughters,2015,7.0,Drama
92458,The Best Years,2020,6.7,Drama
77066,Summertime,2016,6.2,Drama


#### Fixing the Function

In [138]:
indices = pd.Series(processed_data.index, index=processed_data['primaryTitle']).drop_duplicates()

In [151]:
def get_recommendations_the_right_way(title, df, people_matrix, genre_matrix):
    try:
        # 1. Get the LABEL for the movie title
        label_idx = indices[title]
    except KeyError:
        return "Movie not found in the dataset."

    # 2. THE FIX: Get the INTEGER POSITION of that label in the DataFrame's index
    try:
        position_idx = df.index.get_loc(label_idx)
    except KeyError:
        return "Index label not found in DataFrame's current index."

    print(f"\n--- DEBUG: Finding recommendations for '{title}' ---")
    print(f"DEBUG: Found Label: {label_idx}")
    print(f"DEBUG: Translated to Position: {position_idx}")

    # 3. Use the correct INTEGER POSITION to slice the matrices
    people_sim_scores = cosine_similarity(people_matrix[position_idx], people_matrix)[0]
    genre_sim_scores = cosine_similarity(genre_matrix[position_idx], genre_matrix)[0]
    
    # The rest of the logic can now proceed, as it's working on the correct data
    w_people = 0.60 
    w_genre = 0.40
    combined_content_scores = (w_people * people_sim_scores) + (w_genre * genre_sim_scores)

    sim_scores = list(enumerate(combined_content_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:251]
    
    movie_indices = [i[0] for i in sim_scores]
    recs_df = df.iloc[movie_indices].copy()
    
    # Adding back the popularity filter which is a good practice
    m = df['numVotes'].quantile(0.70) # Raised the bar for popularity
    C = df['averageRating'].mean()
    def weighted_rating(x, m=m, C=C):
        v = x['numVotes']
        R = x['averageRating']
        return (v / (v + m) * R) + (m / (v + m) * C)
    recs_df['popularity_score'] = recs_df.apply(weighted_rating, axis=1)

    recs_df['content_score'] = [i[1] for i in sim_scores]
    recs_df['content_score'] = recs_df['content_score'] / recs_df['content_score'].max()
    recs_df['popularity_score'] = recs_df['popularity_score'] / recs_df['popularity_score'].max()
    
    recs_df['final_score'] = (0.7 * recs_df['content_score']) + (0.3 * recs_df['popularity_score'])
    
    final_recs = recs_df.sort_values('final_score', ascending=False).head(20)
    
    return final_recs[['primaryTitle', 'startYear', 'averageRating', 'genres']]

In [164]:
get_recommendations_the_right_way('Men in Black: International', processed_data, people_tfidf_matrix, genre_tfidf_matrix)


--- DEBUG: Finding recommendations for 'Men in Black: International' ---
DEBUG: Found Label: 60829
DEBUG: Translated to Position: 60818


,primaryTitle,startYear,averageRating,genres
46979,Lift,2024,5.5,"Action,Comedy,Crime"
14401,Friday,1995,7.2,"Comedy,Drama"
40472,Law Abiding Citizen,2009,7.4,"Action,Crime,Drama"
71695,Thor: Ragnarok,2017,7.9,"Action,Adventure,Comedy"
15127,Set It Off,1996,6.9,"Action,Crime,Drama"
15912,The Negotiator,1998,7.3,"Action,Crime,Drama"
20774,The Italian Job,2003,7.0,"Action,Crime,Thriller"
19386,A Man Apart,2003,6.0,"Action,Crime,Drama"
77013,The Fate of the Furious,2017,6.6,"Action,Crime,Thriller"
36183,Thor: Love and Thunder,2022,6.2,"Action,Adventure,Comedy"


#### Two Step Function Recommendation

In [163]:
processed_data['title_year'] = processed_data['primaryTitle'].astype(str) + " (" + processed_data['startYear'].astype(str) + ")"
unique_indices = pd.Series(processed_data.index, index=processed_data['title_year']).drop_duplicates()
unique_indices

title_year
The Story of the Kelly Gang (1906)                                      0
Dante's Inferno (1911)                                                  1
Passion (1919)                                                          2
Fantômas: In the Shadow of the Guillotine (1913)                        3
Ingeborg Holm (1913)                                                    4
                                                                    ...  
Time for Farewell! Pudding's One Last Request! (2019)               93541
The World in Shock! The Fifth Emperor of the Sea Arrives! (2019)    93542
Salt City (India) (2022)                                            93543
Il talento del calabrone (2020)                                     93544
Coven (2020)                                                        93545
Length: 93493, dtype: int64

In [165]:
def find_movie(title, df):
    """
    Finds all movies that match a given title.
    If only one match is found, return its unique 'title_year' identifier.
    If multiple matches are found, return a list of their 'title_year' identifiers for disambiguation.
    If no matches are found, return None.
    """
    # Find all rows where the primaryTitle matches
    matches = df[df['primaryTitle'] == title]
    
    if len(matches) == 0:
        return None
    elif len(matches) == 1:
        # If there's only one, we can proceed directly
        return matches.iloc[0]['title_year']
    else:
        # If there are multiple, return the list of unique IDs for the user to choose from
        return matches['title_year'].tolist()

In [190]:
def get_recommendations_by_id(unique_id, df, people_matrix, genre_matrix):
    try:
        label_idx = unique_indices[unique_id]
        position_idx = df.index.get_loc(label_idx)
    except KeyError:
        return f"Error: The movie ID '{unique_id}' was not found."

    # The rest of the logic is the same and now guaranteed to be safe
    people_sim_scores = cosine_similarity(people_matrix[position_idx], people_matrix)[0]
    genre_sim_scores = cosine_similarity(genre_matrix[position_idx], genre_matrix)[0]
    
    w_people = 0.75
    w_genre = 0.25
    combined_content_scores = (w_people * people_sim_scores) + (w_genre * genre_sim_scores)
    # ... (rest of the function from before)
    
    sim_scores = list(enumerate(combined_content_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:251]
    movie_indices = [i[0] for i in sim_scores]
    recs_df = df.iloc[movie_indices].copy()
    
    m = df['numVotes'].quantile(0.70)
    C = df['averageRating'].mean()
    def weighted_rating(x, m=m, C=C):
        v = x['numVotes']
        R = x['averageRating']
        return (v / (v + m) * R) + (m / (v + m) * C)
    recs_df['popularity_score'] = recs_df.apply(weighted_rating, axis=1)

    recs_df['content_score'] = [i[1] for i in sim_scores]
    recs_df['content_score'] = recs_df['content_score'] / recs_df['content_score'].max()
    recs_df['popularity_score'] = recs_df['popularity_score'] / recs_df['popularity_score'].max()
    recs_df['final_score'] = (0.7 * recs_df['content_score']) + (0.3 * recs_df['popularity_score'])
    final_recs = recs_df.sort_values('final_score', ascending=False).head(20)
    
    return final_recs[['primaryTitle', 'startYear', 'averageRating', 'genres']]

In [170]:
search_term = "Superman"
search_results = find_movie(search_term, processed_data)

print(f"--- Searching for '{search_term}' ---")
if isinstance(search_results, list):
    print("Found multiple matches! Please choose one:")
    for movie_id in search_results:
        print(f"  - {movie_id}")
elif search_results:
    print(f"Found one match: {search_results}. Getting recommendations...")
    # Since we found one, we can call the recommendation function directly
    print(get_recommendations_by_id(search_results, processed_data, people_tfidf_matrix, genre_tfidf_matrix))
else:
    print("No movie found with that title.")

print("\n" + "="*50 + "\n")

# 2. User chooses the 2025 version they had in mind
chosen_movie_id = "Superman (2025)"
print(f"--- Getting recommendations for the specific choice: '{chosen_movie_id}' ---")
get_recommendations_by_id(chosen_movie_id, processed_data, people_tfidf_matrix, genre_tfidf_matrix)

--- Searching for 'Superman' ---
Found multiple matches! Please choose one:
  - Superman (1948)
  - Superman (1978)
  - Superman (1988)
  - Superman (2025)


--- Getting recommendations for the specific choice: 'Superman (2025)' ---


,primaryTitle,startYear,averageRating,genres
23930,Slither,2006,6.5,"Comedy,Horror,Sci-Fi"
43432,It's Cow or Never,2022,9.0,"Action,Adventure,Comedy"
57168,Guardians of the Galaxy,2014,8.0,"Action,Adventure,Comedy"
43430,Murn After Reading,2022,8.7,"Action,Adventure,Comedy"
85118,Guardians of the Galaxy Vol. 3,2023,7.9,"Action,Adventure,Comedy"
43427,Better Goff Dead,2022,8.4,"Action,Adventure,Comedy"
43425,A Whole New Whirled,2022,8.1,"Action,Adventure,Comedy"
73933,Guardians of the Galaxy Vol. 2,2017,7.6,"Action,Adventure,Comedy"
43426,"Best Friends, For Never",2022,8.0,"Action,Adventure,Comedy"
83616,The Suicide Squad,2021,7.2,"Action,Adventure,Comedy"


In [174]:
find_movie("Outer Banks", processed_data)

'Outer Banks (2020)'

In [175]:
get_recommendations_by_id("Outer Banks (2020)", processed_data, people_tfidf_matrix, genre_tfidf_matrix)

,primaryTitle,startYear,averageRating,genres
48937,The Coastal Venture,2021,8.7,"Action,Crime,Drama"
63550,Secret of the Gnomon,2023,8.5,"Action,Crime,Drama"
34893,The Phantom,2020,8.2,"Action,Crime,Drama"
34889,Parcel 9,2020,8.2,"Action,Crime,Drama"
48933,Homecoming,2021,8.2,"Action,Crime,Drama"
48936,Trapped,2021,8.2,"Action,Crime,Drama"
34892,The Bell Tower,2020,8.0,"Action,Crime,Drama"
34888,Midsummers,2020,7.9,"Action,Crime,Drama"
59629,Welcome to Kitty Hawk,2023,8.2,"Action,Crime,Drama"
42452,The Gold,2021,7.8,"Action,Crime,Drama"


- The recommendations are far better now that the the feature weights are tuned properly and the indexing bug in the function is fixed, although for TV Shows the recommendations are all just the episodes of the TV Show itself.
- Also the people_soup seems to be too strict with the personalities

### Approach Three

- This method will populate the people soup with more of the ensamble cast and crew, it will also not use all the `tvEpisode` entries in the `titleType` column

In [176]:
refined_data = processed_data[processed_data['titleType'] != 'tvEpisode'].copy()

In [201]:
def create_refined_people_soup(row):
    """
    Creates a people soup that balances director/lead actor with the wider cast/crew.
    - Directors/Creators get a high weight (3x).
    - Top 2 lead actors get a medium weight (2x).
    - The next 3 actors get a low weight (1x) to capture the ensemble feel.
    - Writers/Showrunners also get a low weight (1x) as they are very important for TV.
    """
    ordered_unique_directors = list(dict.fromkeys(row['directors_list']))
    ordered_unique_actors = list(dict.fromkeys(row['actors_list']))
    ordered_unique_writers = list(dict.fromkeys(row['writers_list']))

    # Apply the new, more balanced weights
    weighted_directors = ordered_unique_directors * 3      
    weighted_lead_actors = ordered_unique_actors[:2] * 2  
    other_actors = ordered_unique_actors[2:9]             
    weighted_writers = ordered_unique_writers * 1         

    # Combine all the weighted parts into the final soup
    soup_list = weighted_directors + weighted_lead_actors + other_actors + weighted_writers
    return ' '.join(soup_list)

In [202]:
# Apply this new function to our refined DataFrame
refined_data['people_soup'] = refined_data.apply(create_refined_people_soup, axis=1)

In [203]:
# Let's check the result for "You" to confirm our fix
you_series_id = "You (2018)" 
# Find the specific row for the TV show "You"
you_row = refined_data[refined_data['title_year'] == you_series_id]
you_row

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,...,director_names,writer_names,feature_soup,genres_list,actors_list,directors_list,writers_list,people_soup,genre_soup,title_year
86615,tt7335184,tvSeries,You,You,0,2018,2025,45,"Crime,Drama,Romance",7.6,...,NaN,NaN,crime drama romance PennBadgley VictoriaPedret...,"[crime, drama, romance]","[PennBadgley, VictoriaPedretti, CharlotteRitch...",[],[],PennBadgley VictoriaPedretti PennBadgley Victo...,crime drama romance,You (2018)


#### Vectorization

In [204]:
print("\n--- Step 3: Rebuilding Models and Index with Refined Data ---")
people_tfidf_refined = TfidfVectorizer(stop_words='english')
people_tfidf_matrix_refined = people_tfidf_refined.fit_transform(refined_data['people_soup'])

genre_tfidf_refined = TfidfVectorizer()
genre_tfidf_matrix_refined = genre_tfidf_refined.fit_transform(refined_data['genre_soup'])


--- Step 3: Rebuilding Models and Index with Refined Data ---


In [205]:
unique_indices_refined = pd.Series(refined_data.index, index=refined_data['title_year']).drop_duplicates()
print("Models and index have been successfully rebuilt.")

Models and index have been successfully rebuilt.


In [212]:
get_recommendations_by_id('Black Widow (2021)', refined_data, people_tfidf_matrix_refined, genre_tfidf_matrix_refined)

,primaryTitle,startYear,averageRating,genres
22308,Somersault,2004,6.7,"Drama,Romance"
75048,Avengers: Endgame,2019,8.4,"Action,Adventure,Sci-Fi"
75047,Avengers: Infinity War,2018,8.4,"Action,Adventure,Sci-Fi"
55345,Captain America: The Winter Soldier,2014,7.7,"Action,Adventure,Sci-Fi"
68422,Jurassic World: Rebirth,2025,6.2,"Action,Adventure,Sci-Fi"
70705,Berlin Syndrome,2017,6.3,"Drama,Horror,Mystery"
71665,Captain America: Civil War,2016,7.8,"Action,Sci-Fi"
62348,Avengers: Age of Ultron,2015,7.3,"Action,Adventure,Sci-Fi"
22800,The Island,2005,6.8,"Action,Sci-Fi,Thriller"
31987,The Avengers,2012,8.0,"Action,Sci-Fi"


In [ ]:
# --- The Final, Final, *Corrected* Version of the Recommendation Function ---

def get_recommendations_v_final(title_year, df, people_matrix, genre_matrix, indices_map):
    try:
        label_idx = indices_map[title_year]
        position_idx = df.index.get_loc(label_idx)
    except KeyError:
        return f"Error finding '{title_year}'. Please use 'Title (Year)' format and ensure it's in the dataset."

    # --- Content Score Calculation ---
    people_sim_scores = cosine_similarity(people_matrix[position_idx], people_matrix)[0]
    genre_sim_scores = cosine_similarity(genre_matrix[position_idx], genre_matrix)[0]
    
    w_people = 0.75 
    w_genre = 0.25
    combined_content_scores = (w_people * people_sim_scores) + (w_genre * genre_sim_scores)

    sim_scores = list(enumerate(combined_content_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:501]
    
    # --- THE FIX IS HERE: A clear, sequential process ---
    
    # 1. Create two separate lists from the sim_scores.
    movie_indices = [i[0] for i in sim_scores]
    content_scores_list = [i[1] for i in sim_scores]

    # 2. Build the DataFrame. Its length will match the lists.
    recs_df = df.iloc[movie_indices].copy()
    
    # 3. Assign the content_scores list directly. The lengths are guaranteed to match now.
    recs_df['content_score'] = content_scores_list
    
    # --- The rest of the scoring logic can now proceed safely ---
    m = df['numVotes'].quantile(0.70)
    C = df['averageRating'].mean()
    def weighted_rating(x, m=m, C=C):
        v = x['numVotes']
        R = x['averageRating']
        return (v / (v + m) * R) + (m / (v + m) * C)
    recs_df['popularity_score'] = recs_df.apply(weighted_rating, axis=1)

    max_year = df['startYear'].max()
    min_year = df['startYear'].min()
    recs_df['recency_score'] = (recs_df['startYear'] - min_year) / (max_year - min_year)
    
    # Normalize scores
    recs_df['content_score'] = recs_df['content_score'] / recs_df['content_score'].max()
    recs_df['popularity_score'] = recs_df['popularity_score'] / recs_df['popularity_score'].max()
    
    w_content = 0.60
    w_popularity = 0.25
    w_recency = 0.15

    recs_df['final_score'] = (w_content * recs_df['content_score']) + \
                             (w_popularity * recs_df['popularity_score']) + \
                             (w_recency * recs_df['recency_score'])
    
    final_recs = recs_df.sort_values('final_score', ascending=False).head(20)
    
    return final_recs[['primaryTitle', 'startYear', 'averageRating', 'genres']]






In [227]:
get_recommendations_v_final('The Godfather (1972)', refined_data, people_tfidf_matrix_refined, genre_tfidf_matrix_refined, unique_indices_refined)

,primaryTitle,startYear,averageRating,genres
6976,The Godfather Part II,1974,9.0,"Crime,Drama"
12163,The Godfather Part III,1990,7.6,"Crime,Drama"
9539,The Outsiders,1983,7.0,"Crime,Drama"
8201,Apocalypse Now,1979,8.4,"Drama,Mystery,War"
15665,The Rainmaker,1997,7.2,"Crime,Drama,Thriller"
9568,Rumble Fish,1983,7.1,"Crime,Drama,Romance"
31414,The Godfather: A Novel for Television,1977,9.5,"Crime,Drama,Thriller"
5716,The Rain People,1969,6.8,Drama
9768,The Cotton Club,1984,6.6,"Crime,Drama,Music"
33342,Tetro,2009,6.8,Drama


In [237]:
get_recommendations_v_final("Glass Onion (2022)", refined_data, people_tfidf_matrix_refined, genre_tfidf_matrix_refined, unique_indices_refined)

,primaryTitle,startYear,averageRating,genres
91165,Knives Out,2019,7.9,"Comedy,Crime,Drama"
22645,Brick,2005,7.1,"Crime,Drama,Mystery"
31906,The Brothers Bloom,2008,6.7,"Action,Adventure,Comedy"
42438,Looper,2012,7.4,"Action,Drama,Sci-Fi"
63321,Star Wars: Episode VIII - The Last Jedi,2017,6.9,"Action,Adventure,Fantasy"
22458,Motherless Brooklyn,2019,6.8,"Crime,Drama,Mystery"
15825,American History X,1998,8.5,"Crime,Drama"
39376,Only Murders in the Building,2021,8.1,"Comedy,Crime,Drama"
38876,Leaves of Grass,2009,6.4,"Comedy,Crime,Drama"
19385,Death to Smoochy,2002,6.3,"Comedy,Crime,Drama"


In [234]:
find_movie("Outer Banks", refined_data)

'Outer Banks (2020)'

### Saving the Assets

In [238]:
import joblib
import os

# Create a directory to store our model assets if it doesn't exist
output_dir = 'saved_model'
os.makedirs(output_dir, exist_ok=True)

# 1. Save the final, clean DataFrame
joblib.dump(refined_data, os.path.join(output_dir, 'movies_df.pkl'))
print("Saved: movies_df.pkl")

# 2. Save the People TF-IDF matrix
joblib.dump(people_tfidf_matrix_refined, os.path.join(output_dir, 'people_tfidf_matrix.pkl'))
print("Saved: people_tfidf_matrix.pkl")

# 3. Save the Genre TF-IDF matrix
joblib.dump(genre_tfidf_matrix_refined, os.path.join(output_dir, 'genre_tfidf_matrix.pkl'))
print("Saved: genre_tfidf_matrix.pkl")

# 4. Save the unique indices map
joblib.dump(unique_indices_refined, os.path.join(output_dir, 'indices_map.pkl'))
print("Saved: indices_map.pkl")

print(f"\nAll model assets have been successfully saved to the '{output_dir}' directory.")

Saved: movies_df.pkl
Saved: people_tfidf_matrix.pkl
Saved: genre_tfidf_matrix.pkl
Saved: indices_map.pkl

All model assets have been successfully saved to the 'saved_model' directory.


In [240]:
print(f"pandas version: {pd.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"joblib version: {joblib.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {scipy.__version__}")

pandas version: 2.2.2
scikit-learn version: 1.5.1
joblib version: 1.4.2
numpy version: 1.26.4
scipy version: 1.13.1
